In [1]:
import DataHelpers
import AugmentationHelpers
import InferenceHelpers
import EnsembleHelpers
import TrainingHelpers
import ModelHelpers
import os
import torch
import pandas as pd
#Directory containing the test images of DFUC 2021
TEST_DATA_DIR = "./data/test/"
#Directory to save trained models
MODEL_DIR = "./Ensemble_Models/"
#Directory containing the training images of DFUC 2021
TRAIN_DATA_DIR = "./data/train/images/"
#.csv file containing the labels of the training dataset
TRAIN_LABELS_FILE=os.path.abspath("./data/train/train.csv")


In [2]:
#File with model parameters
MODEL_PARAMETERS_FILE="./Hyperparameters_Ensemble_Model_1+2+3+4.csv"

In [4]:
model_parameters=pd.read_csv(MODEL_PARAMETERS_FILE,sep=";")
for indexi, settings in model_parameters.iterrows():
    MODEL_DIR_SUB=MODEL_DIR+"/ID"+str(settings["ID"])+"/"
    
    MODEL_NAME=settings["Model"]
    PRETRAINED=settings["Pretrained"]
    learning_rate_warm_up=settings["Learning rate warm up"]
    learning_rate=settings["Learning rate training"]
    use_scheduler=settings["Scheduler"]
    BATCH_SIZE = settings["Batch size"]
    IMAGE_SIZE = settings["Image size"]
    augmentation=settings["Augmentations"]
    NUM_EPOCHS_WARMUP=settings["Num epochs warm up"]
    NUM_EPOCHS=settings["Num epochs"]
    use_oversampling=settings["Oversampling"]
    optimizer_type=settings["Optimizer"]
    drop_rate=settings["Drop rate"]
    epochs=NUM_EPOCHS_WARMUP+NUM_EPOCHS
    multi_GPU=settings["Multi GPU"]
    model_file=MODEL_DIR_SUB+"/Model_Epoch_"+str(epochs)+".pth"
    if os.path.exists(model_file):
        device = ("cuda" if torch.cuda.is_available() else "cpu")
        torch.manual_seed(103)
        trainingDataset=DataHelpers.prepare_Training_Dataset(TRAIN_LABELS_FILE,TRAIN_DATA_DIR)
        NUM_CLASSES=DataHelpers.get_number_of_classes_in_dataset(trainingDataset)
        model=ModelHelpers.load_inference_model(MODEL_NAME,PRETRAINED,NUM_CLASSES,drop_rate,device,model_file,multi_GPU)
        InferenceHelpers.inference_Model(device,TEST_DATA_DIR,augmentation,IMAGE_SIZE,model,MODEL_DIR_SUB+"/results.csv",10)